In [ ]:

from tensorflow import keras
from keras.layers import Dense
from tensorflow.keras.layers import Activation
from keras.layers import BatchNormalization
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Flatten
from keras.layers import Input
from tensorflow.keras.layers import Conv2D
from keras.models import Model
from tensorflow.keras.layers import LeakyReLU, PReLU
from keras.layers import add
from keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Softmax
#from tensorflow.keras.layers import AveragePooling2D
#import tensorflow_addons as tfa

# Residual block
def res_block_gen(model, kernal_size, filters, strides, initializer):

    gen = model

    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same", kernel_initializer=initializer)(model)
    model = BatchNormalization(momentum = 0.5)(model)
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = filters, kernel_size = kernal_size, strides = strides, padding = "same", kernel_initializer=initializer)(model)
    model = BatchNormalization(momentum = 0.5)(model)

    model = add([gen, model])

    return model


# Network Architecture is same as given in Paper https://arxiv.org/pdf/1609.04802.pdf
class Generator(object):

    def __init__(self, noise_shape):

        self.noise_shape = noise_shape

    def generator(self):
        init = RandomNormal(stddev=0.02)

        gen_input = Input(shape = self.noise_shape)
        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(gen_input)
        model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)

        gen_model = model

        # Using 16 Residual Blocks
        for index in range(1):
	        model = res_block_gen(model, 3, 64, 1, init)


        model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model)
        model = BatchNormalization(momentum = 0.5)(model)
        model = add([gen_model, model])

        model = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model)

        # Task1 for classification
        model1= Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model)
        model1= Conv2D(filters = 4, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model1)
        output1=Softmax()(model1)

        # Task2 for downscaling with 3 upsampling blocks
        model2 = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model)
        model2 = UpSampling2D(size = 2)(model2)
        model2 = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model2)

        model2 = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model2)
        model2 = UpSampling2D(size = 3)(model2)
        model2 = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model2)

        model2 = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = "same", kernel_initializer=init)(model2)
        model2 = UpSampling2D(size = 2)(model2)
        model2 = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model2)

        output2 = Conv2D(filters = 1, kernel_size = 9, strides = 1, padding = "same", kernel_initializer=init)(model2)
	#    model = Activation('tanh')(model)

        generator_model = Model(inputs = gen_input, outputs = [output1, output2])

        return generator_model

# model_gen=Generator((13, 16, 1)).generator()
# model_gen.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model_gen)

In [1]:
! pip freeze numpy

absl-py==1.4.0
aiohttp==3.9.1
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.3.1
altair==4.2.2
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array-record==0.5.0
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.0
attrs==23.1.0
audioread==3.0.1
autograd==1.6.2
Babel==2.13.1
backcall==0.2.0
beautifulsoup4==4.11.2
bidict==0.22.1
bigframes==0.15.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.1
bqplot==0.12.42
branca==0.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.2
catalogue==2.0.10
certifi==2023.11.17
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.7
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.27.7
cmdstanpy==1.2.0
colorcet==3.0.1
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.0
cryptography==41.0.7
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.2
cvxpy==1.3.2
cycler==0.12.1
c

In [ ]:
from Network import Generator
from keras.models import Model
import numpy as np
from keras.layers import Input
from tensorflow.keras.optimizers import Adam
import numpy as np
from tqdm import tqdm
from numpy import save
from numpy import load
import tensorflow.keras.backend as K
import tensorflow as tf
#from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
from tensorflow.keras.utils import to_categorical
a = 1
# def my_MSE_weighted(y_true, y_pred):
#   weights= tf.clip_by_value(y_true, K.log(0.1+1), K.log(100.0+1))
#   return K.mean(tf.multiply(weights, tf.abs(tf.subtract(y_pred, y_true))))

n_classes=4
def weighted_categorical_crossentropy(weights):
    weights=weights.reshape((1,1,1,n_classes))
    def wcce(y_true, y_pred):
        Kweights=K.constant(weights)
        y_true=K.cast(y_true,y_pred.dtype)


        return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)


    return wcce

class_weights=np.array([4, 19, 23, 56]) # inverse percentage of classes
class_loss = weighted_categorical_crossentropy(weights=class_weights)

def make_FSS_loss(mask_size):  # choose any mask size for calculating densities

    def my_FSS_loss(y_true, y_pred):

        # First: DISCRETIZE y_true and y_pred to have only binary values 0/1
        # (or close to those for soft discretization)
        want_hard_discretization = False

        # This example assumes that y_true, y_pred have the shape (None, N, N, 1).
        a = y_true.simpy()
        cutoff = 0.5  # choose the cut off value for discretization

        if (want_hard_discretization):
           # Hard discretization:
           # can use that in metric, but not in loss
           y_true_binary = tf.where(y_true>cutoff, 1.0, 0.0)
           y_pred_binary = tf.where(y_pred>cutoff, 1.0, 0.0)

        else:
           # Soft discretization
           c = 10 # make sigmoid function steep
           y_true_binary = tf.math.sigmoid( c * ( y_true - cutoff ))
           y_pred_binary = tf.math.sigmoid( c * ( y_pred - cutoff ))

        # Done with discretization.

        # To calculate densities: apply average pooling to y_true.
        # Result is O(mask_size)(i,j) in Eq. (2) of [RL08].
        # Since we use AveragePooling, this automatically includes the factor 1/n^2 in Eq. (2).
        pool1 = tf.keras.layers.AveragePooling2D(pool_size=(mask_size, mask_size), strides=(1, 1),
           padding='same')
        y_true_density = pool1(y_true_binary);
        # Need to know for normalization later how many pixels there are after pooling
        n_density_pixels = tf.cast( (tf.shape(y_true_density)[1] * tf.shape(y_true_density)[2]) ,
           tf.float32 )

        # To calculate densities: apply average pooling to y_pred.
        # Result is M(mask_size)(i,j) in Eq. (3) of [RL08].
        # Since we use AveragePooling, this automatically includes the factor 1/n^2 in Eq. (3).
        pool2 = tf.keras.layers.AveragePooling2D(pool_size=(mask_size, mask_size),
                                                 strides=(1, 1), padding='same')
        y_pred_density = pool2(y_pred_binary);

        # This calculates MSE(n) in Eq. (5) of [RL08].
        # Since we use MSE function, this automatically includes the factor 1/(Nx*Ny) in Eq. (5).
        MSE_n = tf.keras.losses.MeanSquaredError()(y_true_density, y_pred_density)

        # To calculate MSE_n_ref in Eq. (7) of [RL08] efficiently:
        # multiply each image with itself to get square terms, then sum up those terms.

        # Part 1 - calculate sum( O(n)i,j^2
        # Take y_true_densities as image and multiply image by itself.
        O_n_squared_image = tf.keras.layers.Multiply()([y_true_density, y_true_density])
        # Flatten result, to make it easier to sum over it.
        O_n_squared_vector = tf.keras.layers.Flatten()(O_n_squared_image)
        # Calculate sum over all terms.
        O_n_squared_sum = tf.reduce_sum(O_n_squared_vector)

        # Same for y_pred densitites:
        # Multiply image by itself
        M_n_squared_image = tf.keras.layers.Multiply()([y_pred_density, y_pred_density])
        # Flatten result, to make it easier to sum over it.
        M_n_squared_vector = tf.keras.layers.Flatten()(M_n_squared_image)
        # Calculate sum over all terms.
        M_n_squared_sum = tf.reduce_sum(M_n_squared_vector)

        MSE_n_ref = (O_n_squared_sum + M_n_squared_sum) / n_density_pixels

        # FSS score according to Eq. (6) of [RL08].
        # FSS = 1 - (MSE_n / MSE_n_ref)

        # FSS is a number between 0 and 1, with maximum of 1 (optimal value).
        # In loss functions: We want to MAXIMIZE FSS (best value is 1),
        # so return only the last term to minimize.

        # Avoid division by zero if MSE_n_ref == 0
        # MSE_n_ref = 0 only if both input images contain only zeros.
        # In that case both images match exactly, i.e. we should return 0.
        my_epsilon = tf.keras.backend.epsilon()  # this is 10^(-7)

        if (want_hard_discretization):
           if MSE_n_ref == 0:
              return( MSE_n )
           else:
              return( MSE_n / MSE_n_ref )
        else:
           return (MSE_n / (MSE_n_ref + my_epsilon) )

    return my_FSS_loss

mask_size = 3


image_shape_hr = (24,36,1)
image_shape_lr = (2,3,5) # coarse input
downscale_factor = 12
# load low resolution other variables data for training
# load low resolution data for training reforecast
PATH = '/scratch/users/nus/e0560091/data/' # CHANGE TO YOUR OWN PATH
reforecast_train=load('X_train_ensemble.npy')

# load high resolution data for training WRF
yhr_train=load('S:\Climate SRGAN\srganMedium-main\Model3\y_hr_train (6).npy')

#load low resolution data for validation
reforecast_val=load('X_val_ensemble.npy')

#load high resolution data for validation
yhr_val=load('S:\Climate SRGAN\srganMedium-main\Model3\y_hr_val (6).npy')

# load stage4 classified image for training

reanalysis_class_train=load('S:\Climate SRGAN\srganMedium-main\Model3\y_class_train (6).npy')
reanalysis_class_train = np.clip(np.round(reanalysis_class_train).astype(int), 0, 4 - 1)
reanalysis_class_train_vector=reanalysis_class_train.reshape(-1,)
reanalysis_class_val=load('S:\Climate SRGAN\srganMedium-main\Model3\y_class_val (6).npy')
# convert stage4_class to categorical variable
unique_values = np.unique(reanalysis_class_train)
print(unique_values)
reanalysis_class_train=to_categorical(reanalysis_class_train, num_classes=n_classes)
reanalysis_class_val=to_categorical(reanalysis_class_val, num_classes=n_classes)

#****************************************************************************************

def train(epochs, batch_size):

    x_train_lr=reforecast_train
    y_train_hr=yhr_train

    x_val_lr=reforecast_val
    y_val_hr=yhr_val

    x_train_lr=reforecast_train
    y_train_class=reanalysis_class_train
    y_train_hr=yhr_train

    x_val_lr=reforecast_val
    y_val_class=reanalysis_class_val
    y_val_hr=yhr_val

    batch_count = int(x_train_lr.shape[0] / batch_size)

    generator = Generator(image_shape_lr).generator()
    generator.compile(loss=[class_loss, make_FSS_loss(mask_size)], optimizer = Adam(learning_rate=0.0001, beta_1=0.9), loss_weights=[0.01, 1.0],metrics=['mae', 'mse'])
    loss_file = open('losses.txt' , 'w+')
    loss_file.close()

    for e in range(1, epochs+1):

        print ('-'*15, 'Epoch %d' % e, '-'*15)

        for _ in tqdm(range(batch_count)):

            rand_nums = np.random.randint(0, x_train_lr.shape[0], size=batch_size)

            x_lr = x_train_lr[rand_nums]
            y_hr = y_train_hr[rand_nums]
            y_class=y_train_class[rand_nums]

            gen_loss=generator.train_on_batch(x_lr, [y_class,y_hr])
        gen_loss = str(gen_loss)
        #from here
        gen_loss, y_pred = generator.train_on_batch(x_lr, [y_class, y_hr])

        # Save the values of y_pred to a file
        with open('y_pred_values.txt', 'a') as file:
            file.write('epoch%d : y_pred values = %s\n' % (e, str(y_pred))) #here
        val_loss = generator.evaluate(x_val_lr, [y_val_class, y_val_hr], verbose=0)
        val_loss = str(val_loss)
        loss_file = open('losses.txt' , 'a')
        loss_file.write('epoch%d : generator_loss = %s; validation_loss = %s\n'
                        %(e, gen_loss, val_loss))

        loss_file.close()
        if e <=10:
            if e  % 5== 0:
                generator.save('gen_model%d.h5' % e)
        else:
             if e  % 10 == 0:
                generator.save('gen_model%d.h5' % e)



train(5 , 64)





ModuleNotFoundError: ignored